In [2]:
import pandas as pd
import torch
from sklearn.utils import shuffle
import torch.nn as nn

In [3]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
import pandas as pd
rnames = ['userId', 'movieId', 'rating', "TimeStamp"]
ratings = pd.read_table("ml-1m.inter", header=0, names=rnames, engine='python')
ratings

,userId,movieId,rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [5]:
ratio = 0.8
ratings = shuffle(ratings)
# ratings
train = ratings.copy()
test = ratings.copy()
train.iloc[int(ratio*len(ratings)):,2] = None
# train.iloc[int(ratio*len(ratings)):,'rating'] = 0
test.iloc[:int(ratio*len(ratings)),2] = None

In [6]:
ratings.describe()

,userId,movieId,rating,TimeStamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [7]:
rating_matrix = train.pivot(index='userId', columns='movieId', values='rating')
n_users, n_movies = rating_matrix.shape
# Scaling ratings to between 0 and 1, this helps our model by constraining predictions
min_rating, max_rating = train['rating'].min(), train['rating'].max()
rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)
print(rating_matrix)
print(n_users*n_movies-rating_matrix.isnull().values.sum())

movieId  1     2     3     4     5     6     7     8     9     10    ...   
userId                                                               ...   
1         1.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  \
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   NaN   NaN   NaN  0.25   NaN   NaN   NaN   NaN  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
6036      NaN   NaN   NaN  0.25   NaN  0.50   NaN   NaN   NaN   NaN  ...   
6037      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6038      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6039      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6040      0.5   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieId  39

In [8]:
# Replacing missing ratings with -1 so we can filter them out later

rating_matrix[rating_matrix.isnull()] = -1
rating_matrix = torch.FloatTensor(rating_matrix.values).to(device)
print(rating_matrix)

tensor([[ 1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        ...,
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [ 0.5000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],
       device='cuda:0')


In [9]:
# class GMF(torch.nn.Module):
#     def __init__(self, feature_size):
#         super().__init__()
#         self.fc = nn.Linear(feature_size, feature_size)
#         self.out = nn.Linear(feature_size, 1)
#         self.ReLU = nn.ReLU()

#     def forward(self, matrix, u_features, v_features):
#         x = torch.einsum('ik,jk->ijk',[u_features, v_features])
#         x = self.fc(x)
#         x = self.ReLU(x)
#         x = self.out(x)
#         predicted_rating = torch.sigmoid(x)
#         # non_zero_mask = (matrix != -1).type(torch.FloatTensor)
#         # predicted_rating = torch.sigmoid(torch.mm(u_features, v_features.t()))
        
#         return predicted_rating

In [10]:
# A = torch.tensor([[1,2,3]])
# B = torch.tensor([[1,2,3]])
# print(torch.einsum('ik,jk->ijk',[A, B]))

In [14]:
class Fed_NCF(nn.Module):
    def __init__(self,user_num, item_num, factor_num):
        super(Fed_NCF,self).__init__()
        self.embed_user_GMF = nn.Embedding(user_num,factor_num)
        self.embed_item_GMF = nn.Embedding(item_num,factor_num)
        self.hidden_layer = nn.Linear(factor_num,factor_num)
        self.predict_layer = nn.Linear(factor_num,1)
        self._init_weight_()
        self.ReLU = nn.ReLU()
    def _init_weight_(self):
        nn.init.normal_(self.embed_item_GMF.weight,std=0.01)
        nn.init.normal_(self.embed_user_GMF.weight,std=0.01)

    def forward(self,user,item):
        embed_user_GMF = self.embed_user_GMF(user)
        embed_item_GMF = self.embed_item_GMF(item)
        output_GMF = torch.einsum('ik,jk->ijk',[embed_user_GMF, embed_item_GMF])
        # output_GMF = self.hidden_layer(output_GMF)
        # output_GMF = self.ReLU(output_GMF)
        prediction = torch.sigmoid(self.predict_layer(output_GMF))

        return torch.squeeze(prediction)

In [15]:
class GMFLoss(torch.nn.Module):
    def __init__(self, lam_u=0.3, lam_v=0.3):
        super().__init__()
        self.lam_u = lam_u
        self.lam_v = lam_v
    
    def forward(self, matrix, predicted):
        non_zero_mask = (matrix != -1).type(torch.FloatTensor).to(device)
        diff = (matrix - predicted)**2
        prediction_error = torch.sum(diff*non_zero_mask)
        
        return prediction_error

In [17]:
num_epoch = 100
latent_vectors = 20
gmf = Fed_NCF(n_users, n_movies, latent_vectors).to(device)
gmfloss = GMFLoss(lam_u=0.1, lam_v=0.1)
user = torch.LongTensor(range(n_users)).to(device)
item = torch.LongTensor(range(n_movies)).to(device)

optimizer = torch.optim.Adam(gmf.parameters(), lr=0.01)
for step, epoch in enumerate(range(num_epoch)):
    optimizer.zero_grad()
    # pred = gmf(torch.ones(n_users).long().to(device),torch.ones(n_movies).long().to(device))
    pred = gmf(user,item)
    loss = gmfloss(rating_matrix, pred)
    loss.backward()
    optimizer.step()
    if step % 10 == 0:
        print(f"Step {step}, {loss:.3f}")

In [ ]:
test_rating_matrix = test.pivot(index='userId', columns='movieId', values='rating')

test_rating_matrix[test_rating_matrix.isnull()] = -1
test_rating_matrix = torch.FloatTensor(test_rating_matrix.values).to(device)
test_rating_matrix

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]], device='cuda:0')

In [ ]:
## test FedNCF
non_zero_mask = (test_rating_matrix != -1).type(torch.FloatTensor).to(device)
num = torch.sum(non_zero_mask)
user = torch.LongTensor(range(n_users)).to(device)
item = torch.LongTensor(range(n_movies)).to(device)
predicted_ratings = gmf(user, item)
predicted_ratings = (predicted_ratings*(max_rating - min_rating) + min_rating)*non_zero_mask
actual_ratings = test_rating_matrix*non_zero_mask

AE_diff = torch.abs(predicted_ratings - actual_ratings)
SE_diff = (predicted_ratings - actual_ratings)**2

test_MAE = torch.sum(AE_diff)/num
test_RMSE = torch.sqrt(torch.sum(SE_diff)/num)
print('test_MAE =', test_MAE.data.cpu().numpy())
print('test_RMSE =', test_RMSE.data.cpu().numpy())